### Install liten
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ liten
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/liten
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/liten/dist/liten-0.0.1-py3-none-any.whl
```

Import Apache arrow

In [1]:
import pyarrow as pa
from pyarrow import csv

Import Liten-ten is local rten is remote. rten imports pyarrow library as well.

In [2]:
import liten as ten

<frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.6 of module 'liten._liten' does not match runtime version 3.8
<frozen importlib._bootstrap>:219: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject


In [3]:
import liten.rcliten as rten

Import Ray to be used as a cluster

In [4]:
import ray

Start a cluster with single worker.

In [5]:
ray.init(num_cpus=1)

2021-07-26 22:16:05,886	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.27.237.243',
 'raylet_ip_address': '172.27.237.243',
 'redis_address': '172.27.237.243:6379',
 'object_store_address': '/tmp/ray/session_2021-07-26_22-16-05_188877_1362/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-07-26_22-16-05_188877_1362/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-07-26_22-16-05_188877_1362',
 'metrics_export_port': 40120,
 'node_id': 'eb018d9139b333b05c0312ed33e4fa46cac682b6'}

In [6]:
ray.cluster_resources()

{'object_store_memory': 42.0,
 'memory': 123.0,
 'CPU': 1.0,
 'node:172.27.237.243': 1.0}

Create a Liten Cache Actor. It is residing on a remote node, and being executed on that node. tc is the Liten Cache actor handle.

In [7]:
rten.RCLiten = ray.remote(rten.RCLiten)
tc = rten.RCLiten.remote()

(pid=1458) <frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.6 of module 'liten._liten' does not match runtime version 3.8
(pid=1458) <frozen importlib._bootstrap>:219: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject
(pid=1458) WARNING: Logging before InitGoogleLogging() is written to STDERR
(pid=1458) I20210726 22:16:14.485251  1458 TCache.cpp:24] Created a new TCache


These are fact and dimension tables of TPCH. Read them remotely.

In [8]:
fact_tables = ['lineitem']
dim_tables = ['customer','orders','supplier','nation','region']
tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/'

In [9]:
def read_tables(tables, table_type):
    arrow_tables = []
    for table_name in tables:
        tpch_table = tpch_dir+table_name+'.tbl'
        print('Reading ', tpch_table)
        tc.set_table.remote(table_name, table_type)
        pytable = tc.read_csv.remote(input_file=tpch_table, parse_options=csv_options)
        # print(' Rows=', pytable.num_rows,' Cols=', pytable.num_columns)
        arrow_tables.append(pytable)
    return arrow_tables

In [10]:
%%time
csv_options = pa.csv.ParseOptions(delimiter='|')
pa_fact_tables = read_tables(fact_tables, 1)
pa_dim_tables = read_tables(dim_tables, 0)

Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/lineitem.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/customer.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/orders.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/supplier.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/nation.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/region.tbl
CPU times: user 11.9 ms, sys: 22.9 ms, total: 34.8 ms
Wall time: 19.5 ms


(pid=1458) I20210726 22:16:26.249943  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.249975  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.250736  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.322634  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.322664  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.322723  1458 TCatalog.cpp:17] Created a new TCatalog


(pid=1458) Added Table= b'lineitem'
(pid=1458) Added Table= b'customer'


(pid=1458) I20210726 22:16:26.959901  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.959933  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.960117  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.975929  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.975958  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.975998  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.983350  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.983371  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.983402  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.987301  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.987322  1458 TCatalog.cpp:17] Created a new TCatalog
(pid=1458) I20210726 22:16:26.987352  1458 TCatalog.cpp:17] Created a new TCatalog


(pid=1458) Added Table= b'orders'
(pid=1458) Added Table= b'supplier'
(pid=1458) Added Table= b'nation'
(pid=1458) Added Table= b'region'


In [11]:
tc.info.remote()

ObjectRef(fafba2bafaed5dc3df5a1a820100000001000000)

(pid=1458) I20210726 22:17:07.673210  1458 TConfigs.cpp:16] Created a new TConfigs
(pid=1458) I20210726 22:17:07.673278  1458 TCatalog.cpp:17] Created a new TCatalog


Read a table into TCache

In [12]:
%%time
result = tc.make_dtensor.remote()

CPU times: user 532 µs, sys: 236 µs, total: 768 µs
Wall time: 417 µs


Read Arrow table

In [13]:
result = tc.query6.remote()

(pid=1458)  TPCH QUERY 6 
(pid=1458) SELECT 
(pid=1458)   SUM(L_EXTENDEDPRICE * L_DISCOUNT) AS REVENUE 
(pid=1458) FROM 
(pid=1458)   LINEITEM
(pid=1458) WHERE
(pid=1458)   L_SHIPDATE >= DATE '1997-01-01'
(pid=1458)   AND L_SHIPDATE < DATE '1997-01-01' + INTERVAL '1' YEAR
(pid=1458)   AND L_DISCOUNT BETWEEN 0.07 - 0.01 AND 0.07 + 0.01
(pid=1458)   AND L_QUANTITY < 25;
(pid=1458) 
(pid=1458) Revenue= 156594095.60960016
(pid=1458) 


In [14]:
result = tc.query5.remote()

(pid=1458)  
(pid=1458) SELECT
(pid=1458) 	N_NAME,
(pid=1458) 	SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS REVENUE
(pid=1458) FROM
(pid=1458) 	CUSTOMER,
(pid=1458) 	ORDERS,
(pid=1458) 	LINEITEM,
(pid=1458) 	SUPPLIER,
(pid=1458) 	NATION,
(pid=1458) 	REGION
(pid=1458) WHERE
(pid=1458) 	C_CUSTKEY = O_CUSTKEY
(pid=1458) 	AND L_ORDERKEY = O_ORDERKEY
(pid=1458) 	AND L_SUPPKEY = S_SUPPKEY
(pid=1458) 	AND C_NATIONKEY = S_NATIONKEY
(pid=1458) 	AND S_NATIONKEY = N_NATIONKEY
(pid=1458) 	AND N_REGIONKEY = R_REGIONKEY
(pid=1458) 	AND R_NAME = 'EUROPE'
(pid=1458) 	AND O_ORDERDATE >= DATE '1995-01-01'
(pid=1458) 	AND O_ORDERDATE < DATE '1995-01-01' + INTERVAL '1' YEAR
(pid=1458) GROUP BY
(pid=1458) 	N_NAME
(pid=1458) ORDER BY
(pid=1458) 	REVENUE DESC;
(pid=1458) 
(pid=1458) b'RUSSIA' = 91847.279
(pid=1458) b'UNITED KINGDOM' = 90496.0902
(pid=1458) b'ROMANIA' = 23206.976400000003
(pid=1458) b'FRANCE' = 71966.62169999999
(pid=1458) b'GERMANY' = 147844.0068
(pid=1458) 


This will kill remote Liten Cache.

In [15]:
ray.kill(tc)

Shut down ray now

In [16]:
ray.shutdown()